In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from fastai.vision.all import *


In [2]:


class PhysicsInformedLoss(nn.Module):
    def __init__(self, bem_theory_fn, parametric_model_fn, lambda_betz=0.1, lambda_bem=0.1, lambda_param=0.1):
        super(PhysicsInformedLoss, self).__init__()
        self.bem_theory_fn = bem_theory_fn  # function or model to compute BEM prediction
        self.parametric_model_fn = parametric_model_fn  # function to get parametric model prediction
        self.lambda_betz = lambda_betz  # Betz constraint weight
        self.lambda_bem = lambda_bem    # BEM theory constraint weight
        self.lambda_param = lambda_param  # Parametric model constraint weight
    
    def forward(self, cp_pred, cp_actual, inputs):
        # Standard data-driven loss (MSE between prediction and actual Cp)
        mse_loss = nn.MSELoss()(cp_pred, cp_actual)
        
        # Betz's limit penalty: Encourage Cp to be below Betz limit (0.593)
        betz_penalty = torch.mean(torch.relu(cp_pred - 0.593))
        
        # BEM theory penalty: Penalize deviations from BEM theory
        cp_bem = self.bem_theory_fn(inputs)  # BEM-predicted Cp
        bem_penalty = nn.MSELoss()(cp_pred, cp_bem)
        
        # Parametric model penalty: Penalize deviations from parametric model
        cp_param = self.parametric_model_fn(inputs)  # Parametric-predicted Cp
        param_penalty = nn.MSELoss()(cp_pred, cp_param)
        
        # Total loss
        loss = mse_loss + self.lambda_betz * betz_penalty + self.lambda_bem * bem_penalty + self.lambda_param * param_penalty
        return loss


In [3]:
def bem_theory_fn(inputs):
    # Example: simple formula based on inputs, or a neural network model if complex
    # inputs could be: wind_speed, rotor_speed, etc.
    # This should return a tensor of cp values according to BEM theory
    wind_speed, rotor_speed = inputs[:, 0], inputs[:, 1]
    cp_bem = 0.5 * (wind_speed - rotor_speed)  # Simplified example
    return cp_bem

def parametric_model_fn(inputs):
    # Similar to bem_theory_fn, this could be a simple function or a model.
    wind_speed, rotor_speed = inputs[:, 0], inputs[:, 1]
    cp_param = 0.4 * (wind_speed ** 2) - 0.3 * rotor_speed  # Simplified example
    return cp_param


In [ ]:
class CpPredictionModel(nn.Module):
    def __init__(self, input_dim):
        super(CpPredictionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [ ]:
# Create your dataset (example: X and y tensors for inputs and actual Cp values)
dls = DataLoaders.from_dsets(TensorDataset(X_train, y_train), TensorDataset(X_valid, y_valid), bs=64)

# Instantiate the model
model = CpPredictionModel(input_dim=X_train.shape[1])

# Define the custom loss function with BEM and parametric functions
loss_func = PhysicsInformedLoss(bem_theory_fn, parametric_model_fn, lambda_betz=0.1, lambda_bem=0.2, lambda_param=0.3)

# Set up the FastAI Learner
learn = Learner(dls, model, loss_func=loss_func, metrics=rmse)

# Train the model
learn.fit_one_cycle(10, lr_max=1e-3)
